In [1]:
import requests
import base64
import datetime
from urllib.parse import urlencode

In [2]:
client_id='a2db305d6a8a4ebd8191989f7f400119'
client_secret = 'b0f792e64a1d4572906992c6148d319c'

# Create class functions 

In [3]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_status = True     # True if not expired and False if expiried
    client_id = None 
    client_secret = None 
    token_url= "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs): 
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_creditials(self): 
        """
            reuturns a base 64 encoded string././
        """
        self.client_id = client_id
        self.client_secret = client_secret
        
        if client_secret == None or client_id == None : 
            raise Exception("You must set client id and client sceret") 
        client_creds= f'{client_id}:{client_secret}'
        client_creds_b64=base64.b64encode(client_creds.encode())
        
        return client_creds_b64
    
    def get_token_headers(self): 
        client_creds_b64 = self.get_client_creditials()
        return { 
                "Authorization" : f'Basic {client_creds_b64.decode()}'  # <base64 encoded client_id:client_secret > 
                }
    
    def get_token_data(self):
        return {  
                'grant_type' : 'client_credentials'
                }
    
    def get_resource_header(self):
        access_token=self.get_access_token() 
        headers={
            "Authorization" : f"Bearer {access_token}"   
            }
        return headers
    
    def get_resource(self,lookup_id,resource_type='albums',version='v1'):
        endpoint=f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers=self.get_resource_header()
        r=requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            return {'No results'}
        return r.json()
    
    def get_album(self,_id): 
        return self.get_resource(_id,resource_type='albums')
    
    def get_artist(self,_id):
            return self.get_resource(_id,resource_type='artists')
    
    def get_playlist(self,_id):
        access_token = self.get_access_token()
        playlist_url = f'https://api.spotify.com/v1/playlists/{_id}'
        headers={
                    "Authorization" : f"Bearer {access_token}"   
                    }

        response=requests.get(playlist_url, headers=headers)

        return response #self.get_resource(_id,resource_type='playlist')
        
    def get_access_token(self):
        token=self.access_token
        expiry_time=self.access_token_expires
        current_time=datetime.datetime.now() 
        if expiry_time < current_time:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
            
    def perform_auth(self):
        token_url=self.token_url
        token_data=self.get_token_data()
        token_headers=self.get_token_headers() 
        
        r=requests.post(token_url,data=token_data, headers=token_headers) 
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            raise Exception("Could not authenticate client")
        
        data=r.json()
        current_datetime=datetime.datetime.now() 
        access_token=data['access_token']
        expires_in=data['expires_in'] # seconds
        expires_time_left = current_datetime + datetime.timedelta(seconds=expires_in)
        self.access_token=access_token
        self.access_token_expires = expires_time_left
        self.access_token_expires_status = expires_time_left < current_datetime
        
        return True
    
    def base_search(self,query_params):
        access_token=self.get_access_token() 
        headers=self.get_resource_header()
        endpoint='https://api.spotify.com/v1/search'
        lookup_url=f'{endpoint}?{query_params}'
        r=requests.get(lookup_url ,headers=headers)
        if r.status_code not in range(200,299):         # returns a boolean value. checking http stauts reponse 
            return {'No results'}
        return r.json()
    
    def search(self, query=None,search_type='artists'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query,dict):
            query=" ".join([f'{k}:{v}' for k,v in query.items()])
        query_params=urlencode({"q": query, "type" : search_type.lower()})
        return self.base_search(query_params)

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [27]:
spotify.get_playlist('4GBXXvhMRpTpIxw6ekgRzM').json()['tracks']['items'][0]['track'] #['album']['artists'][0]['name']

{'album': {'album_type': 'compilation',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3yY2gUcIsjMr8hjo51PoJ8'},
    'href': 'https://api.spotify.com/v1/artists/3yY2gUcIsjMr8hjo51PoJ8',
    'id': '3yY2gUcIsjMr8hjo51PoJ8',
    'name': 'The Smiths',
    'type': 'artist',
    'uri': 'spotify:artist:3yY2gUcIsjMr8hjo51PoJ8'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'HK',
   'HN',
   'HR',
   'HT',
   'HU',
   'ID',
   'IE',
   'IL',
   '

In [13]:
def get_spotify_links(emotion):
    
    client_id='a2db305d6a8a4ebd8191989f7f400119'
    client_secret = '60456ef0016e4762b59deab63c1c05e3'
    
    spotify = SpotifyAPI(client_id, client_secret)
    
    playlist_ids = {'sad': '4GBXXvhMRpTpIxw6ekgRzM',
                   'angry': '7g3ldE2qrypfcaCMuGr1uO',
                   'happy': '2w1JTNzciFssDDVmbUVLS0',
                   'disgust': '5GX0cpDTiyqK7VNZpA55Wc'}

    spotify_urls=[]
    spotify_tracknames = []
    for i in range(5):
        spotify_urls.append(spotify.get_playlist(playlist_ids[emotion]).json()['tracks']['items'][i]['track']['external_urls']['spotify'])
        spotify_tracknames.append(spotify.get_playlist(playlist_ids[emotion]).json()['tracks']['items'][i]['track']['name']) 
    return spotify_tracknames, spotify_urls

In [17]:
get_spotify_links('happy')

['https://open.spotify.com/track/0gzqZ9d1jIKo9psEIthwXe', 'https://open.spotify.com/track/1v2zyAJrChw5JnfafSkwkJ', 'https://open.spotify.com/track/6JWddKPdqvDc2WkPEi9grC', 'https://open.spotify.com/track/5SiHgGiBYZDgYSx2hLdrIb', 'https://open.spotify.com/track/5hhF25sWnZvXhimOHLjTRE']
['Beautiful Day', 'Shiny Happy People', 'Have A Nice Day', 'Here Comes the Sun', 'I Feel Fine - Remastered 2015']


(['Beautiful Day',
  'Shiny Happy People',
  'Have A Nice Day',
  'Here Comes the Sun',
  'I Feel Fine - Remastered 2015'],
 ['https://open.spotify.com/track/0gzqZ9d1jIKo9psEIthwXe',
  'https://open.spotify.com/track/1v2zyAJrChw5JnfafSkwkJ',
  'https://open.spotify.com/track/6JWddKPdqvDc2WkPEi9grC',
  'https://open.spotify.com/track/5SiHgGiBYZDgYSx2hLdrIb',
  'https://open.spotify.com/track/5hhF25sWnZvXhimOHLjTRE'])